<a href="https://colab.research.google.com/github/gauravgund/gauravgund/blob/master/wav2vec_fairseq__finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/mailong25/self-supervised-speech-recognition


In [1]:
%cd /content


/content


In [2]:
#Uninstall the current CUDA version
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

#Download CUDA 10.0
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
#install CUDA kit dpkg
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [1]:
# !apt-get --purge remove cuda nvidia* libnvidia-*
# !dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
# !apt-get remove cuda-*
# !apt autoremove
# !apt-get update

In [2]:
# !wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
# #install CUDA kit dpkg
# !dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
# !sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
# !apt-get update
# !apt-get install cuda-10-0

In [3]:
!git clone https://github.com/mailong25/self-supervised-speech-recognition.git
%cd self-supervised-speech-recognition

Cloning into 'self-supervised-speech-recognition'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (236/236), done.
remote: Total 686 (delta 121), reused 44 (delta 6), pack-reused 443
Receiving objects: 100% (686/686), 12.91 MiB | 10.57 MiB/s, done.
Resolving deltas: 100% (300/300), done.
/content/self-supervised-speech-recognition


In [4]:
!mkdir libs
%cd libs

/content/self-supervised-speech-recognition/libs


In [5]:
import torch
torch.cuda.is_available()
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [6]:
%%bash
pip install soundfile torchaudio sentencepiece editdistance sklearn
#If cuda version < 11 (eg. cuda 10.1):
pip install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#If cuda >= 11:
#pip install torch==1.6.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.6.0+cu101 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.6.0+cu101 which is incompatible.
ERROR: torchaudio 0.8.1 has requirement torch==1.8.1, but you'll have torch 1.6.0+cu101 which is incompatible.


In [7]:
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!git checkout c8a0659be5cdc15caa102d5bbf72b872567c4859

!pip install --editable ./
%cd ..

Cloning into 'fairseq'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 22015 (delta 4), reused 13 (delta 3), pack-reused 21978
Receiving objects: 100% (22015/22015), 10.19 MiB | 26.03 MiB/s, done.
Resolving deltas: 100% (16423/16423), done.
/content/self-supervised-speech-recognition/libs/fairseq
Note: checking out 'c8a0659be5cdc15caa102d5bbf72b872567c4859'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at c8a0659b Stronger --checkpoint-activations test (#1505)
Obtaining file:///content/self-supervised-speech-recognition/

In [8]:
!sudo apt-get update && sudo apt-get -y install apt-utils libpq-dev libsndfile-dev
!sudo apt install libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
!sudo apt-get install libsndfile1-dev libopenblas-dev libfftw3-dev libgflags-dev libgoogle-glog-dev

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [9]:
%%bash
git clone https://github.com/kpu/kenlm.git
cd kenlm
mkdir -p build && cd build
cmake ..
make -j 4
cd ../..

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Boost version: 1.65.1
-- Found the following Boost libraries:


Cloning into 'kenlm'...


In [9]:
# !apt-get update;
# !wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64 -O cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
# !dpkg -i cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
# !apt-key add /var/cuda-repo-10-0-local/7fa2af80.pub
# !apt-get update
# !apt-get -y install gcc-7 g++-7
# !apt-get -y install cuda 

# !export PATH=/usr/local/cuda/bin${PATH:+:${PATH}}
# !export LD_LIBRARY_PATH=/usr/local/cuda/lib64\${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}

# # check if installed successfully
# !/usr/local/cuda/bin/nvcc --version

# # install python-nvcc plugin
# !pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
# %load_ext nvcc_plugin


# # # you can start writing your cuda code
# # %%cu
# # #include <iostream>
# # int main(){ 
# #     std::cout << "hello" << std::endl;
# #     return 0;
# # }

In [10]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [11]:
%%bash
rm -rf wav2letter
git clone -b v0.2 https://github.com/facebookresearch/wav2letter.git
cd wav2letter/bindings/python
export KENLM_ROOT_DIR=/content/self-supervised-speech-recognition/libs/kenlm
export USE_CUDA=1
pip install --log pip_log -e .
cd ../../..

Obtaining file:///content/self-supervised-speech-recognition/libs/wav2letter/bindings/python
  Running setup.py develop for wav2letter


Cloning into 'wav2letter'...


Training

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
%cd /content/
!wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec_small.pt

/content
--2021-03-31 05:00:34--  https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec_small.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 950500491 (906M) [application/octet-stream]
Saving to: ‘wav2vec_small.pt’

wav2vec_small.pt    100%[===================>] 906.47M  64.9MB/s    in 14s     

2021-03-31 05:00:48 (65.4 MB/s) - ‘wav2vec_small.pt’ saved [950500491/950500491]



In [13]:
!python3 /content/self-supervised-speech-recognition/gen_dict.py --transcript_file /content/gdrive/MyDrive/wav2vec_fairseq/transcript.txt --save_dir /content/gdrive/MyDrive/wav2vec_fairseq


In [16]:
%cd /content/
! mkdir config
! mkdir config/finetuning

/content


In [20]:
%cd /content/self-supervised-speech-recognition
! python3 /content/self-supervised-speech-recognition/finetune.py --transcript_file /content/gdrive/MyDrive/wav2vec_fairseq/transcript.txt --pretrain_model /content/wav2vec_small.pt --dict_file /content/gdrive/MyDrive/wav2vec_fairseq/dict.ltr.txt


/content/self-supervised-speech-recognition
100% 945/945 [00:00<00:00, 945.37it/s]
fairseq-hydra-train task.data=/content/self-supervised-speech-recognition/manifest distributed_training.distributed_world_size=1 +optimization.update_freq='[24]' model.w2v_path=/content/wav2vec_small.pt dataset.num_workers=4 dataset.max_tokens=2800000 --config-dir config/finetuning --config-name base_1h
2021-03-31 05:26:18 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 200, 'log_format': 'json', 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'mo

In [17]:
!python3 /content/self-supervised-speech-recognition/train_lm.py --kenlm_path /content/self-supervised-speech-recognition/libs/kenlm --transcript_file /content/gdrive/MyDrive/wav2vec_fairseq/transcript.txt --additional_file /content/gdrive/MyDrive/wav2vec_fairseq/text_corpus.txt --ngram 3 --output_path /content/gdrive/MyDrive/wav2vec_fairseq/model_checkpoint


100% 1890/1890 [00:00<00:00, 255280.80it/s]
3874
3874
=== 1/5 Counting and sorting n-grams ===
Reading /content/gdrive/MyDrive/wav2vec_fairseq/model_checkpoint/world_lm_data.train
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 3284467712 bytes == 0x563002a1e000 @  0x7f3a8de471e7 0x562fc67487a2 0x562fc67377ca 0x562fc6738208 0x562fc66c2308 0x562fc66ae066 0x7f3a8bfe0bf7 0x562fc66afbaa
****************************************************************************************************
Unigram tokens 34580 types 3877
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:46524 2:1493885440 3:2801035264
tcmalloc: large alloc 2801041408 bytes == 0x562fc7e6c000 @  0x7f3a8de471e7 0x562fc67487a2 0x562fc67377ca 0x562fc6738208 0x562fc66c28d7 0x562fc66ae066 0x7f3a8bfe0bf7 0x562fc66afbaa
tcmalloc: large alloc 1493893120 bytes == 0x56306edc4000 @  0x7f3a8de471e7 0x562fc67487a2 0x562fc67377ca 0x562fc6738208 0x562fc

In [27]:
%cd /content/self-supervised-speech-recognition/libs/fairseq
!pip install --editable ./

/content/self-supervised-speech-recognition/libs/fairseq
Obtaining file:///content/self-supervised-speech-recognition/libs/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Found existing installation: fairseq 1.0.0a0+c8a0659
    Can't uninstall 'fairseq'. No files were found to uninstall.
  Running setup.py develop for fairseq


In [47]:
# import os
# os.listdir('/content/self-supervised-speech-recognition/outputs/2021-03-31/05-26-18/checkpoints')
# import shutil
# shutil.copy('/content/self-supervised-speech-recognition/outputs/2021-03-31/05-26-18/checkpoints/checkpoint_best.pt', '/content/gdrive/MyDrive/wav2vec_fairseq/model_checkpoint')

'/content/gdrive/MyDrive/wav2vec_fairseq/model_checkpoint/checkpoint_best.pt'

In [48]:
%cd /content/self-supervised-speech-recognition/libs
import fairseq

/content/self-supervised-speech-recognition/libs


In [57]:
! pwd

/content/self-supervised-speech-recognition


In [18]:
%cd /content/self-supervised-speech-recognition
from stt import Transcriber


/content/self-supervised-speech-recognition


ImportError: ignored